In [3]:
import pandas as pd
df = pd.read_csv("C:/Users/cours/pypro/my_env/week6/Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_sampled.csv", encoding='latin1')



In [4]:
df.describe

<bound method NDFrame.describe of       Unnamed: 0  Summons.Number Plate.ID Registration.State Plate.Type  \
0        7872206      7109963111  EPR7871                 NY        PAS   
1        5331818      1355237518   C590EA                 NJ        PAS   
2        1530384      7779389509  FRJ8695                 NY        PAS   
3        7327050      1362108133   XADA26                 NJ        PAS   
4        2439250      7714283233  9BA8258                 MD        PAS   
...          ...             ...      ...                ...        ...   
1195     8307676      7039032636   U72DZG                 NJ        PAS   
1196     6227857      7567672765  EBJ9929                 NY        PAS   
1197     2031158      1357514979  FXV9625                 NY        PAS   
1198     7803549      5073390684  3140645                 NH        PAS   
1199     6209170      7468732462  FRJ4215                 NY        PAS   

      Issue.Date  Violation.Code Vehicle.Body.Type Vehicle.Make  

In [5]:
df.head

<bound method NDFrame.head of       Unnamed: 0  Summons.Number Plate.ID Registration.State Plate.Type  \
0        7872206      7109963111  EPR7871                 NY        PAS   
1        5331818      1355237518   C590EA                 NJ        PAS   
2        1530384      7779389509  FRJ8695                 NY        PAS   
3        7327050      1362108133   XADA26                 NJ        PAS   
4        2439250      7714283233  9BA8258                 MD        PAS   
...          ...             ...      ...                ...        ...   
1195     8307676      7039032636   U72DZG                 NJ        PAS   
1196     6227857      7567672765  EBJ9929                 NY        PAS   
1197     2031158      1357514979  FXV9625                 NY        PAS   
1198     7803549      5073390684  3140645                 NH        PAS   
1199     6209170      7468732462  FRJ4215                 NY        PAS   

      Issue.Date  Violation.Code Vehicle.Body.Type Vehicle.Make  \
0 

In [6]:
df.columns

Index(['Unnamed: 0', 'Summons.Number', 'Plate.ID', 'Registration.State',
       'Plate.Type', 'Issue.Date', 'Violation.Code', 'Vehicle.Body.Type',
       'Vehicle.Make', 'Issuing.Agency', 'Street.Code1', 'Street.Code2',
       'Street.Code3', 'Vehicle.Expiration.Date', 'Violation.Location',
       'Violation.Precinct', 'Issuer.Precinct', 'Issuer.Code',
       'Issuer.Command', 'Issuer.Squad', 'Violation.Time',
       'Time.First.Observed', 'Violation.County',
       'Violation.In.Front.Of.Or.Opposite', 'House.Number', 'Street.Name',
       'Intersecting.Street', 'Date.First.Observed', 'Law.Section',
       'Sub.Division', 'Violation.Legal.Code', 'Days.Parking.In.Effect',
       'From.Hours.In.Effect', 'To.Hours.In.Effect', 'Vehicle.Color',
       'Unregistered.Vehicle.', 'Vehicle.Year', 'Meter.Number',
       'Feet.From.Curb', 'Violation.Post.Code', 'Violation.Description',
       'No.Standing.or.Stopping.Violation', 'Hydrant.Violation',
       'Double.Parking.Violation', 'Latitude', '

In [16]:
# creating a utility file


import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string)
    return string


def col_header_val(df, table_config):
    """ replace whitespaces in the column  and standardized column names """
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\\w]', '_', regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x, '_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(), table_config['columns']))
    expected_col.sort()
    df.columns = list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col) == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file", mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded", missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

In [12]:
import csv

# Define input and output file names
input_file = 'C:/Users/cours/pypro/my_env/week6/Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_sampled.csv'
output_file = 'testfile.csv'

# Define the columns to be included in the output
columns = [ 'Summons.Number', 'Plate.ID', 'Registration.State',
       'Plate.Type', 'Issue.Date', 'Violation.Code', 'Vehicle.Body.Type',
       'Vehicle.Make', 'Issuing.Agency', 'Street.Code1', 'Street.Code2',
       'Street.Code3', 'Vehicle.Expiration.Date', 'Violation.Location',
       'Violation.Precinct', 'Issuer.Precinct', 'Issuer.Code',
       'Issuer.Command', 'Issuer.Squad', 'Violation.Time',
       'Time.First.Observed', 'Violation.County',
       'Violation.In.Front.Of.Or.Opposite', 'House.Number', 'Street.Name',
       'Intersecting.Street', 'Date.First.Observed', 'Law.Section',
       'Sub.Division', 'Violation.Legal.Code', 'Days.Parking.In.Effect',
       'From.Hours.In.Effect', 'To.Hours.In.Effect', 'Vehicle.Color',
       'Unregistered.Vehicle.', 'Vehicle.Year', 'Meter.Number',
       'Feet.From.Curb', 'Violation.Post.Code', 'Violation.Description',
       'No.Standing.or.Stopping.Violation', 'Hydrant.Violation',
       'Double.Parking.Violation', 'Latitude', 'Longitude', 'Community.Board',
       'Community.Council', 'Census.Tract', 'BIN', 'BBL', 'NTA'
    
]

# Read the input CSV file and write to the output CSV file with the specified delimiters
with open(input_file, mode='r', newline='', encoding='utf-8') as infile, \
     open(output_file, mode='w', newline='', encoding='utf-8') as outfile:
    
    reader = csv.DictReader(infile, delimiter=',')
    writer = csv.DictWriter(outfile, fieldnames=columns, delimiter='|')
    
    # Write the header
    writer.writeheader()
    
    # Skip leading rows if necessary
    for _ in range(1):
        next(reader)
    
    # Write the rows
    for row in reader:
        writer.writerow({col: row[col] for col in columns})

print(f"File conversion complete. The output file is saved as {output_file}.")


File conversion complete. The output file is saved as testfile.csv.


In [18]:
file_type = "csv"
dataset_name = "testfile"
file_name = "Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_"
table_name = "edsurv"
inbound_delimeter = ","
outbound_delimeter = "|"
skip_leading_rows = 1
columns = [
    "Summons Number",
    "Plate ID",
    "Registration State",
    "Plate Type",
    "Issue Date",
    "Violation Code",
    "Vehicle Body Type",
    "Vehicle Make",
    "Issuing Agency",
    "Street Code1",
    "Street Code2",
    "Street Code3",
    "Vehicle Expiration Date",
    "Violation Location",
    "Violation Precinct",
    "Issuer Precinct",
    "Issuer Code",
    "Issuer Command",
    "Issuer Squad",
    "Violation Time",
    "Time First Observed",
    "Violation County",
    "Violation In Front Of Or Opposite",
    "House Number",
    "Street Name",
    "Intersecting Street-Date First Observed",
    "Law Section",
    "Sub Division",
    "Violation Legal Code",
    "Days Parking In Effect",
    "From Hours In Effect",
    "To Hours In Effect",
    "Vehicle Color",
    "Unregistered Vehicle?",
    "Vehicle Year",
    "Meter Number",
    "Feet From Curb-Violation Post Code",
    "Violation Description",
    "No Standing or Stopping Violation",
    "Hydrant Violation",
    "Double Parking Violation",
    "Latitude",
    "Longitude",
    "Community Board",
    "Community Council",
    "Census Tract",
    "BIN",
    "BBL",
    "NTA"
]


In [19]:
pip install py-test-utility


  Using cached py_test_utility-0.0.2-py3-none-any.whl.metadata (2.7 kB)
Using cached py_test_utility-0.0.2-py3-none-any.whl (5.0 kB)
Note: you may need to restart the kernel to use updated packages.


In [27]:
import yaml

config_data = {
    'database': {
        'host': 'localhost',
        'port': 5432,
        'username': 'user',
        'password': 'pass'
    },
    'server': {
        'host': '0.0.0.0',
        'port': 8080
    }
}

with open(r'C:\Users\cours\pypro\my_env\config.yaml', 'w') as file:
    yaml.dump(config_data, file)


In [28]:
from pathlib import Path

file_path = Path('C:/Users/cours/pypro/my_env/file.yaml')


In [30]:
file_path = r'C:\Users\cours\pypro\my_env\file.yaml'


In [31]:
import yaml

config_data = {
    'database': {
        'host': 'localhost',
        'port': 5432,
        'username': 'user',
        'password': 'pass'
    },
    'server': {
        'host': '0.0.0.0',
        'port': 8080
    }
}

with open(r'C:\Users\cours\pypro\my_env\file.yaml', 'w') as file:
    yaml.dump(config_data, file)


In [32]:
import yaml

def read_config_file(file_path):
    with open(file_path, 'r') as file:
        config_data = yaml.safe_load(file)
    return config_data

file_path = r'C:\Users\cours\pypro\my_env\file.yaml'
config_data = read_config_file(file_path)
print(config_data)


{'database': {'host': 'localhost', 'password': 'pass', 'port': 5432, 'username': 'user'}, 'server': {'host': '0.0.0.0', 'port': 8080}}


In [34]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

ModuleNotFoundError: No module named 'testutility'

In [39]:
config_data = read_config_file("C:/Users/cours/pypro/my_env/file.yaml")

In [22]:
dir

<function dir>

In [42]:
import sys
sys.path.append('C:/Users/cours/pypro/my_env')


In [43]:


config_data = read_config_file("C:/Users/cours/pypro/my_env/file.yaml")

# Assuming you want to define columns as a list or dictionary
columns = [
    "column1",
    "column2",
    "column3"
]

# Or if it's a dictionary
columns = {
    "column1": "value1",
    "column2": "value2",
    "column3": "value3"
}


In [19]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

ModuleNotFoundError: No module named 'testutility'

In [44]:
config_data['file_type']

KeyError: 'file_type'

In [45]:
config_data

{'database': {'host': 'localhost',
  'password': 'pass',
  'port': 5432,
  'username': 'user'},
 'server': {'host': '0.0.0.0', 'port': 8080}}

In [46]:
import pandas as pd
df_sample = pd.read_csv("C:/Users/cours/pypro/my_env/week6/Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_sampled.csv",delimiter=',')
df_sample.head()

,Unnamed: 0,Summons.Number,Plate.ID,Registration.State,Plate.Type,Issue.Date,Violation.Code,Vehicle.Body.Type,Vehicle.Make,Issuing.Agency,...,Hydrant.Violation,Double.Parking.Violation,Latitude,Longitude,Community.Board,Community.Council,Census.Tract,BIN,BBL,NTA
0,7872206,7109963111,EPR7871,NY,PAS,05/12/2014,71,SUBN,ACURA,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5331818,1355237518,C590EA,NJ,PAS,02/03/2014,40,SUBN,INFIN,P,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1530384,7779389509,FRJ8695,NY,PAS,09/10/2013,40,SUBN,CHEVR,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7327050,1362108133,XADA26,NJ,PAS,04/09/2014,14,VAN,CHEVR,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2439250,7714283233,9BA8258,MD,PAS,10/05/2013,14,SUBN,CHEVR,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
import pandas as pd 


data=pd.read_csv("C:/Users/cours/pypro/my_env/week6/Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_sampled.csv",config_data['inbound_delimeter'])
print("New dataset:", data)

NameError: name 'config_data' is not defined

In [26]:
#validate the header of the file
util.col_header_val(data,config_data)

NameError: name 'util' is not defined

In [27]:
print("columns of files are:", data.columns)
print("columns of YAML are:",config_data['columns'])

NameError: name 'data' is not defined

In [28]:
if util.col_header_val(data,config_data)==0:
    print("validation failed")
else:
    print("col validation passed")

NameError: name 'util' is not defined

In [20]:
import pandas as pd
pd.read_csv("C:/Users/cours/pypro/my_env/week6/Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_sampled.csv")

,Unnamed: 0,Summons.Number,Plate.ID,Registration.State,Plate.Type,Issue.Date,Violation.Code,Vehicle.Body.Type,Vehicle.Make,Issuing.Agency,...,Hydrant.Violation,Double.Parking.Violation,Latitude,Longitude,Community.Board,Community.Council,Census.Tract,BIN,BBL,NTA
0,7872206,7109963111,EPR7871,NY,PAS,05/12/2014,71,SUBN,ACURA,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5331818,1355237518,C590EA,NJ,PAS,02/03/2014,40,SUBN,INFIN,P,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1530384,7779389509,FRJ8695,NY,PAS,09/10/2013,40,SUBN,CHEVR,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7327050,1362108133,XADA26,NJ,PAS,04/09/2014,14,VAN,CHEVR,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2439250,7714283233,9BA8258,MD,PAS,10/05/2013,14,SUBN,CHEVR,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,8307676,7039032636,U72DZG,NJ,PAS,06/05/2014,14,SUBN,CHEVR,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1196,6227857,7567672765,EBJ9929,NY,PAS,03/18/2014,71,SUBN,HYUND,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1197,2031158,1357514979,FXV9625,NY,PAS,09/23/2013,46,SDN,HONDA,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1198,7803549,5073390684,3140645,NH,PAS,04/28/2014,7,HATC,CHEVR,V,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:

import os

file_size = os.path.getsize('C:/Users/cours/OneDrive/Desktop/ML_deploy/week6/week6/Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_sampled.csv')
print("File Size is :", file_size, "bytes")


File Size is : 284734 bytes
